# Complete Residual Stream Validation v2

**Purpose:** Validate "Compress-then-Broadcast" pattern with CORRECT residual stream measurement

**Models:**
- Pythia-6.9B (LayerNorm, GELU) - baseline
- Gemma-7B (RMSNorm, GeGLU) - verify expansion
- Apertus-8B (RMSNorm, xIELU) - Paper #2 connection
- **LLaMA-3.1-8B (RMSNorm, SwiGLU)** - NEW: Compare to Mistral (same MLP type!)

**Key Questions:**
1. Does LLaMA-3.1 (SwiGLU) behave like Mistral (SwiGLU)?
2. Is SwiGLU expansion different from GeGLU/xIELU?
3. Cumulative Energy: Does "net inertia" correlate with norm type?

**Reference:** Mistral-7B = 1.37x expansion (RMSNorm + SwiGLU)

In [ ]:
# Cell 1: Imports and Setup
import torch
import numpy as np
import json
import gc
from datetime import datetime
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# For Colab: upgrade transformers for Apertus support (needs v4.56+)
IN_COLAB = False
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running in Google Colab")
    print("Upgrading transformers for Apertus support...")
    import subprocess
    subprocess.run(['pip', 'install', '-q', 'transformers>=4.56.0'], check=True)
    print("Transformers upgraded!")
except:
    pass

# HuggingFace Login via Colab Secrets (same as before)
HF_LOGGED_IN = False
try:
    from huggingface_hub import login
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token)
        HF_LOGGED_IN = True
        print("✅ HuggingFace login successful via Colab secrets!")
    else:
        print("⚠️ HF_TOKEN not found in Colab secrets")
except Exception as e:
    print(f"⚠️ HF login: {e}")

# Check GPU
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"GPU Memory: {gpu_mem:.1f} GB")
else:
    gpu_name = "CPU"
    gpu_mem = 0
    print("WARNING: Running on CPU - models will be very slow!")

In [ ]:
# Cell 2: Model Definitions

MODELS_TO_TEST = {
    # Pythia - LayerNorm baseline
    'pythia-6.9b': {
        'hf_name': 'EleutherAI/pythia-6.9b',
        'params': 6.9e9,
        'layers': 32,
        'family': 'pythia',
        'norm': 'LayerNorm',
        'mlp': 'GELU',
        'memory_gb': 18,
        'paper2_inversion': None,
        'expected_last_gain': 6.3
    },
    # Gemma - RMSNorm + GeGLU
    'gemma-7b': {
        'hf_name': 'google/gemma-7b',
        'params': 7e9,
        'layers': 28,
        'family': 'gemma',
        'norm': 'RMSNorm',
        'mlp': 'GeGLU',
        'memory_gb': 20,
        'paper2_inversion': None,
        'expected_last_gain': 2.52
    },
    # Apertus - RMSNorm + xIELU (Paper #2 model!)
    'apertus-8b': {
        'hf_name': 'swiss-ai/Apertus-8B-2509',
        'params': 8e9,
        'layers': 32,
        'family': 'apertus',
        'norm': 'RMSNorm',
        'mlp': 'xIELU',
        'memory_gb': 22,
        'paper2_inversion': 28,  # L28 is where correlation inverts in Paper #2!
        'expected_last_gain': None
    },
    # LLaMA 3.1 - RMSNorm + SwiGLU (SAME as Mistral! Key comparison!)
    'llama3.1-8b': {
        'hf_name': 'meta-llama/Llama-3.1-8B',
        'params': 8e9,
        'layers': 32,
        'family': 'llama',
        'norm': 'RMSNorm',
        'mlp': 'SwiGLU',
        'memory_gb': 20,
        'paper2_inversion': None,
        'expected_last_gain': 1.37  # Expecting similar to Mistral (same MLP!)
    }
}

# Fallback for lower memory GPUs
FALLBACK_MODELS = {
    'pythia-2.8b': {
        'hf_name': 'EleutherAI/pythia-2.8b',
        'params': 2.8e9,
        'layers': 32,
        'family': 'pythia',
        'norm': 'LayerNorm',
        'mlp': 'GELU',
        'memory_gb': 8,
        'paper2_inversion': None,
        'expected_last_gain': 2.1
    },
    'gemma-2b': {
        'hf_name': 'google/gemma-2b',
        'params': 2e9,
        'layers': 18,
        'family': 'gemma',
        'norm': 'RMSNorm',
        'mlp': 'GeGLU',
        'memory_gb': 6,
        'paper2_inversion': None,
        'expected_last_gain': 13.72
    }
}

# Reference: Mistral results from previous investigation
MISTRAL_REFERENCE = {
    'last_layer_gain': 1.37,
    'initial_explosion': 43.86,
    'plateau_range': (1, 29),
    'pattern': 'Compress-Hold-Explode',
    'family': 'mistral',
    'norm': 'RMSNorm',
    'mlp': 'SwiGLU'  # Same as LLaMA 3.1!
}

# Select models based on GPU memory
def select_models(gpu_mem_gb):
    """Select appropriate models based on available GPU memory."""
    selected = {}
    
    if gpu_mem_gb >= 40:
        # Full test - all 4 models (A100 or equivalent)
        selected = MODELS_TO_TEST.copy()
    elif gpu_mem_gb >= 22:
        # Skip LLaMA 3.1 (can run separately)
        selected['pythia-6.9b'] = MODELS_TO_TEST['pythia-6.9b']
        selected['gemma-7b'] = MODELS_TO_TEST['gemma-7b']
        selected['apertus-8b'] = MODELS_TO_TEST['apertus-8b']
    elif gpu_mem_gb >= 20:
        # Skip Apertus
        selected['pythia-6.9b'] = MODELS_TO_TEST['pythia-6.9b']
        selected['gemma-7b'] = MODELS_TO_TEST['gemma-7b']
        selected['llama3.1-8b'] = MODELS_TO_TEST['llama3.1-8b']
    elif gpu_mem_gb >= 18:
        # Only Pythia and LLaMA
        selected['pythia-6.9b'] = MODELS_TO_TEST['pythia-6.9b']
        selected['llama3.1-8b'] = MODELS_TO_TEST['llama3.1-8b']
    elif gpu_mem_gb >= 8:
        # Use fallbacks
        selected['pythia-2.8b'] = FALLBACK_MODELS['pythia-2.8b']
        selected['gemma-2b'] = FALLBACK_MODELS['gemma-2b']
    else:
        # Minimal
        selected['pythia-2.8b'] = FALLBACK_MODELS['pythia-2.8b']
    
    return selected

# Select models
models_to_run = select_models(gpu_mem)

print(f"\nSelected models for {gpu_mem:.1f} GB GPU:")
for name, info in models_to_run.items():
    inv = f" [Paper #2 inversion @ L{info['paper2_inversion']}]" if info.get('paper2_inversion') else ""
    print(f"  {name}: {info['layers']} layers, {info['norm']}, {info['mlp']}{inv}")

In [ ]:
# Cell 3: Model Loading with Error Handling
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_key, model_info):
    """Load model with robust error handling."""
    hf_name = model_info['hf_name']
    family = model_info['family']
    
    print(f"\nLoading {model_key} ({hf_name})...")
    
    # Use bfloat16 for better numerical stability (especially for Apertus xIELU)
    if torch.cuda.is_available():
        # Check if bfloat16 is supported
        if torch.cuda.is_bf16_supported():
            dtype = torch.bfloat16
            print(f"  Using bfloat16 (best for xIELU)")
        else:
            dtype = torch.float16
            print(f"  Using float16")
    else:
        dtype = torch.float32
    
    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            hf_name, 
            trust_remote_code=True
        )
        
        # Ensure pad token exists
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        # Load model with explicit dtype
        model = AutoModelForCausalLM.from_pretrained(
            hf_name,
            torch_dtype=dtype,
            device_map='auto' if torch.cuda.is_available() else None,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        
        model.eval()
        
        # Verify layer access based on family
        if family == 'pythia':
            n_layers = len(model.gpt_neox.layers)
        elif family in ['mistral', 'llama', 'gemma', 'apertus']:
            n_layers = len(model.model.layers)
        else:
            if hasattr(model, 'model') and hasattr(model.model, 'layers'):
                n_layers = len(model.model.layers)
            elif hasattr(model, 'transformer') and hasattr(model.transformer, 'h'):
                n_layers = len(model.transformer.h)
            else:
                n_layers = model_info['layers']
        
        print(f"  Loaded successfully! ({n_layers} layers detected)")
        
        if family == 'apertus':
            print(f"  Architecture: {model.config.architectures}")
            print(f"  Hidden size: {model.config.hidden_size}")
            print(f"  Model dtype: {next(model.parameters()).dtype}")
        
        return model, tokenizer, dtype
        
    except Exception as e:
        print(f"  FAILED to load: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None

def cleanup_model(model):
    """Properly cleanup model to free GPU memory."""
    if model is not None:
        del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print("  Memory cleaned up.")

In [ ]:
# Cell 4: Residual Stream Analyzer (robust version)

class ResidualStreamAnalyzer:
    """Analyze the residual stream directly, bypassing RMSNorm measurement issues."""
    
    def __init__(self, model, model_info):
        self.model = model
        self.model_info = model_info
        self.hooks = []
        self.residual_norms = []
        self.embedding_norm = None
        
    def _get_layers(self):
        """Get transformer layers for different architectures."""
        family = self.model_info['family']
        
        if family == 'pythia':
            return self.model.gpt_neox.layers
        elif family in ['mistral', 'llama', 'gemma', 'apertus']:
            # Apertus uses same structure as LLaMA-style models
            return self.model.model.layers
        else:
            # Try common patterns
            if hasattr(self.model, 'model') and hasattr(self.model.model, 'layers'):
                return self.model.model.layers
            elif hasattr(self.model, 'transformer') and hasattr(self.model.transformer, 'h'):
                return self.model.transformer.h
            elif hasattr(self.model, 'gpt_neox') and hasattr(self.model.gpt_neox, 'layers'):
                return self.model.gpt_neox.layers
            else:
                raise ValueError(f"Unknown architecture for family: {family}")
    
    def _get_embedding_layer(self):
        """Get embedding layer for different architectures."""
        family = self.model_info['family']
        
        if family == 'pythia':
            return self.model.gpt_neox.embed_in
        elif family in ['mistral', 'llama', 'gemma', 'apertus']:
            # Apertus uses same structure as LLaMA-style models
            return self.model.model.embed_tokens
        else:
            return self.model.get_input_embeddings()
    
    def _make_embedding_hook(self):
        """Hook to capture embedding output norm."""
        def hook(module, args, output):
            with torch.no_grad():
                self.embedding_norm = output.float().norm().item()
        return hook
    
    def _make_layer_hook(self, layer_idx):
        """Create hook that captures residual stream norm AFTER each layer."""
        def hook(module, args, output):
            # Output is (hidden_states, ...) or just hidden_states
            if isinstance(output, tuple):
                hidden = output[0]
            else:
                hidden = output
            
            with torch.no_grad():
                norm = hidden.float().norm().item()
                self.residual_norms.append((layer_idx, norm))
        
        return hook
    
    def register_hooks(self):
        """Register hooks on embedding and each layer output."""
        # Embedding hook
        embed_layer = self._get_embedding_layer()
        h_emb = embed_layer.register_forward_hook(self._make_embedding_hook())
        self.hooks.append(h_emb)
        
        # Layer hooks
        layers = self._get_layers()
        for i, layer in enumerate(layers):
            h = layer.register_forward_hook(self._make_layer_hook(i))
            self.hooks.append(h)
        
        print(f"  Registered {len(self.hooks)} hooks (1 embedding + {len(layers)} layers)")
    
    def remove_hooks(self):
        for h in self.hooks:
            h.remove()
        self.hooks = []
    
    def clear(self):
        self.residual_norms = []
        self.embedding_norm = None
    
    def get_layer_gains(self):
        """Compute gain = norm(layer_i) / norm(layer_{i-1})."""
        gains = []
        norms = []
        
        # Sort by layer index
        sorted_norms = sorted(self.residual_norms, key=lambda x: x[0])
        
        # Add embedding norm as "layer -1" equivalent
        if self.embedding_norm:
            all_norms = [("emb", self.embedding_norm)] + sorted_norms
        else:
            all_norms = sorted_norms
        
        # Compute gains
        for i in range(1, len(all_norms)):
            prev_norm = all_norms[i-1][1]
            curr_norm = all_norms[i][1]
            
            if prev_norm > 1e-8:
                gain = curr_norm / prev_norm
            else:
                gain = 0.0
            
            gains.append(float(gain))
        
        norms = [n for _, n in all_norms]
        
        return gains, norms

In [ ]:
# Cell 5: Run Residual Stream Analysis

all_results = {}

for model_key, model_info in models_to_run.items():
    print(f"\n{'='*60}")
    print(f"Residual Stream Analysis: {model_key}")
    print(f"{'='*60}")
    
    # Load model (now returns dtype too)
    model, tokenizer, model_dtype = load_model(model_key, model_info)
    if model is None:
        print(f"  Skipping {model_key} due to load failure.")
        continue
    
    try:
        # Setup analyzer
        analyzer = ResidualStreamAnalyzer(model, model_info)
        analyzer.register_hooks()
        
        # Test prompt
        prompt = "The capital of France is"
        inputs = tokenizer(prompt, return_tensors='pt')
        
        # Move to GPU if available
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        # Forward pass with autocast for dtype consistency (fixes xIELU fallback issue)
        print(f"  Running forward pass (dtype: {model_dtype})...")
        with torch.no_grad():
            if torch.cuda.is_available() and model_dtype in [torch.float16, torch.bfloat16]:
                # Use autocast to ensure consistent dtypes throughout forward pass
                with torch.amp.autocast(device_type='cuda', dtype=model_dtype):
                    outputs = model(**inputs)
            else:
                outputs = model(**inputs)
        
        # Get gains
        gains, norms = analyzer.get_layer_gains()
        
        if gains and len(gains) > 1:
            # Statistics
            n_layers = len(gains)
            contracting = sum(1 for g in gains if g < 1.0)
            expanding = sum(1 for g in gains if g > 1.0)
            last_gain = gains[-1] if gains else 0
            max_gain = max(gains) if gains else 0
            max_gain_layer = gains.index(max_gain) if gains else -1
            initial_gain = gains[0] if gains else 0
            
            # Find expansion layers
            expansion_layers = [(i, g) for i, g in enumerate(gains) if g > 1.0]
            
            # Check Paper #2 inversion point
            paper2_layer = model_info.get('paper2_inversion')
            paper2_gain = None
            if paper2_layer is not None and paper2_layer < len(gains):
                paper2_gain = gains[paper2_layer]
            
            print(f"\n  Results:")
            print(f"  Layers (gains): {n_layers}")
            print(f"  Embedding Norm: {norms[0]:.2f}")
            print(f"  Initial Gain (Emb→L0): {initial_gain:.2f}x")
            print(f"  Contracting: {contracting}/{n_layers} ({100*contracting/n_layers:.1f}%)")
            print(f"  Expanding: {expanding}/{n_layers} ({100*expanding/n_layers:.1f}%)")
            print(f"  Last Layer Gain: {last_gain:.4f}x {'EXPANDS!' if last_gain > 1.0 else 'contracts'}")
            print(f"  Max Gain: {max_gain:.4f}x at position {max_gain_layer}")
            print(f"  Final Norm: {norms[-1]:.2f}")
            
            # Compare with expected
            expected = model_info.get('expected_last_gain')
            if expected:
                ratio = last_gain / expected
                print(f"  Expected (FFN): {expected:.2f}x, Ratio: {ratio:.2f}")
            
            if paper2_gain is not None:
                print(f"\n  *** Paper #2 Inversion Point (Layer {paper2_layer}): {paper2_gain:.4f}x ***")
            
            # Top expansion layers
            if expansion_layers:
                print(f"\n  Top Expansion Layers (gain > 1):")
                sorted_exp = sorted(expansion_layers, key=lambda x: x[1], reverse=True)[:5]
                for i, g in sorted_exp:
                    marker = " <- PAPER #2 INVERSION!" if paper2_layer is not None and i == paper2_layer else ""
                    print(f"    Position {i}: {g:.4f}x{marker}")
            
            # Store results
            all_results[model_key] = {
                'family': model_info['family'],
                'norm_type': model_info['norm'],
                'mlp_type': model_info['mlp'],
                'n_layers': n_layers,
                'gains': gains,
                'norms': norms,
                'embedding_norm': norms[0] if norms else None,
                'initial_gain': float(initial_gain),
                'last_gain': float(last_gain),
                'last_expands': bool(last_gain > 1.0),
                'max_gain': float(max_gain),
                'max_gain_layer': int(max_gain_layer),
                'contracting_pct': float(100 * contracting / n_layers),
                'expanding_pct': float(100 * expanding / n_layers),
                'expansion_layers': [(int(i), float(g)) for i, g in expansion_layers],
                'paper2_inversion_layer': paper2_layer,
                'paper2_inversion_gain': float(paper2_gain) if paper2_gain is not None else None,
                'expected_last_gain': expected
            }
        else:
            print(f"  WARNING: No valid gains computed!")
        
        # Cleanup hooks
        analyzer.remove_hooks()
        
    except Exception as e:
        print(f"  ERROR during analysis: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Always cleanup model
        cleanup_model(model)

print(f"\n\n{'='*60}")
print(f"Completed residual stream analysis for {len(all_results)} models.")
print(f"{'='*60}")

In [ ]:
# Cell 6: Cross-Model Comparison (with SwiGLU Analysis)

print("="*60)
print("CROSS-MODEL COMPARISON (Residual Stream)")
print("="*60)

# Build comparison data including Mistral reference
comparison_data = {
    'mistral-7b (ref)': {
        'family': 'mistral',
        'norm_type': 'RMSNorm',
        'mlp_type': 'SwiGLU',
        'initial_gain': MISTRAL_REFERENCE['initial_explosion'],
        'last_gain': MISTRAL_REFERENCE['last_layer_gain'],
        'last_expands': True,
        'source': 'Previous Investigation'
    }
}

for name, res in all_results.items():
    comparison_data[name] = {
        'family': res['family'],
        'norm_type': res['norm_type'],
        'mlp_type': res['mlp_type'],
        'initial_gain': res['initial_gain'],
        'last_gain': res['last_gain'],
        'last_expands': res['last_expands'],
        'source': 'This Experiment'
    }

print("\n| Model | Family | Norm | MLP | Initial | Last | Expands? |")
print("|-------|--------|------|-----|---------|------|----------|")

for name, data in comparison_data.items():
    expands = "YES" if data['last_expands'] else "NO"
    print(f"| {name} | {data['family']} | {data['norm_type']} | {data['mlp_type']} | {data['initial_gain']:.2f}x | {data['last_gain']:.2f}x | {expands} |")

# Universal principle test
all_expand = all(d['last_expands'] for d in comparison_data.values())
print(f"\n{'='*60}")
if all_expand:
    print("UNIVERSAL PRINCIPLE CONFIRMED: ALL models expand at last layer!")
else:
    failing = [n for n, d in comparison_data.items() if not d['last_expands']]
    print(f"WARNING: These models do NOT expand: {failing}")

# SwiGLU COMPARISON (Key for Paper #3!)
print(f"\n{'='*60}")
print("SWIGLU COMPARISON (LLaMA-3.1 vs Mistral)")
print("="*60)

swiglu_models = {n: d for n, d in comparison_data.items() if d['mlp_type'] == 'SwiGLU'}
geglu_models = {n: d for n, d in comparison_data.items() if d['mlp_type'] == 'GeGLU'}
gelu_models = {n: d for n, d in comparison_data.items() if d['mlp_type'] == 'GELU'}
xield_models = {n: d for n, d in comparison_data.items() if d['mlp_type'] == 'xIELU'}

print("\nBy MLP Type:")
print("-" * 50)

if swiglu_models:
    print(f"\n🔵 SwiGLU Models: {list(swiglu_models.keys())}")
    for name, data in swiglu_models.items():
        print(f"   {name}: Initial={data['initial_gain']:.2f}x, Last={data['last_gain']:.2f}x")
    
    # Direct comparison if both LLaMA-3.1 and Mistral present
    if 'llama3.1-8b' in swiglu_models and 'mistral-7b (ref)' in swiglu_models:
        llama_gain = swiglu_models['llama3.1-8b']['last_gain']
        mistral_gain = swiglu_models['mistral-7b (ref)']['last_gain']
        diff = abs(llama_gain - mistral_gain)
        ratio = llama_gain / mistral_gain
        
        print(f"\n   *** SWIGLU COMPARISON ***")
        print(f"   LLaMA-3.1-8B Last Gain:  {llama_gain:.4f}x")
        print(f"   Mistral-7B Last Gain:    {mistral_gain:.4f}x")
        print(f"   Difference: {diff:.4f}x")
        print(f"   Ratio: {ratio:.2f}")
        
        if 0.8 <= ratio <= 1.2:
            print(f"\n   ✅ CONSISTENT! Same MLP type → Similar expansion behavior")
        else:
            print(f"\n   ⚠️ DIVERGENT! Same MLP but different expansion (investigate!)")

if geglu_models:
    print(f"\n🔴 GeGLU Models: {list(geglu_models.keys())}")
    for name, data in geglu_models.items():
        print(f"   {name}: Initial={data['initial_gain']:.2f}x, Last={data['last_gain']:.2f}x")

if gelu_models:
    print(f"\n🟢 GELU Models (LayerNorm): {list(gelu_models.keys())}")
    for name, data in gelu_models.items():
        print(f"   {name}: Initial={data['initial_gain']:.2f}x, Last={data['last_gain']:.2f}x")

if xield_models:
    print(f"\n🟠 xIELU Models: {list(xield_models.keys())}")
    for name, data in xield_models.items():
        print(f"   {name}: Initial={data['initial_gain']:.2f}x, Last={data['last_gain']:.2f}x")

# Summary by Norm Type
print(f"\n{'='*60}")
print("SUMMARY BY NORMALIZATION")
print("="*60)

rmsnorm_gains = [d['last_gain'] for n, d in comparison_data.items() if d['norm_type'] == 'RMSNorm']
layernorm_gains = [d['last_gain'] for n, d in comparison_data.items() if d['norm_type'] == 'LayerNorm']

if rmsnorm_gains:
    print(f"\nRMSNorm Models (n={len(rmsnorm_gains)}):")
    print(f"  Last Gains: {[f'{g:.2f}x' for g in rmsnorm_gains]}")
    print(f"  Mean: {np.mean(rmsnorm_gains):.2f}x, Range: [{min(rmsnorm_gains):.2f}x, {max(rmsnorm_gains):.2f}x]")
    print(f"  ALL EXPAND: {'YES' if all(g > 1.0 for g in rmsnorm_gains) else 'NO'}")

if layernorm_gains:
    print(f"\nLayerNorm Models (n={len(layernorm_gains)}):")
    print(f"  Last Gains: {[f'{g:.2f}x' for g in layernorm_gains]}")
    print(f"  Mean: {np.mean(layernorm_gains):.2f}x")
    print(f"  ALL CONTRACT: {'YES' if all(g < 1.0 for g in layernorm_gains) else 'NO'}")

In [ ]:
# Cell 7: Paper #2 Connection Analysis

print("="*60)
print("PAPER #2 <-> PAPER #3 CONNECTION")
print("="*60)

# Check Apertus
apertus_key = 'apertus-8b'
if apertus_key in all_results:
    apertus = all_results[apertus_key]
    
    print("\nPaper #2 Finding (Apertus-8B):")
    print("  - Embedding-Output correlation INVERTS at Layer 28")
    print("  - Before L28: Positive correlation (consensus building)")
    print("  - After L28: Negative correlation (discrimination)")
    
    print("\nPaper #3 Question:")
    print("  - Does Layer 28 show EXPANSION in residual stream?")
    print("  - Is 'inversion' actually 'expansion'?")
    
    l28_gain = apertus.get('paper2_inversion_gain')
    last_gain = apertus['last_gain']
    
    print(f"\nResults:")
    if l28_gain is not None:
        print(f"  Layer 28 Gain: {l28_gain:.4f}x {'(EXPANDS)' if l28_gain > 1.0 else '(contracts)'}")
    print(f"  Last Layer Gain: {last_gain:.4f}x {'(EXPANDS)' if last_gain > 1.0 else '(contracts)'}")
    
    # Analyze expansion profile
    exp_layers = apertus.get('expansion_layers', [])
    if exp_layers:
        exp_indices = [l for l, g in exp_layers]
        print(f"\n  Expansion layers: {exp_indices}")
        print(f"  Strongest expansion: Position {apertus['max_gain_layer']} ({apertus['max_gain']:.2f}x)")
        
        # Check if L28 is in expansion layers
        if 28 in exp_indices or (l28_gain and l28_gain > 1.0):
            print(f"\n  *** HYPOTHESIS CONFIRMED! ***")
            print(f"  The 'inversion point' from Paper #2 IS an expansion point!")
            print(f"  Semantic Inversion = Residual Expansion = Decision Broadcast")
        else:
            print(f"\n  Layer 28 is NOT an expansion point.")
            if last_gain > 1.0:
                print(f"  But expansion still occurs at last layer!")
else:
    print("\nApertus-8B not tested - cannot verify Paper #2 connection.")
    print("(Need GPU with >= 22GB memory)")

In [ ]:
# Cell 8: Visualization

import os
results_dir = '../Results'
os.makedirs(results_dir, exist_ok=True)

n_models = len(all_results)
if n_models == 0:
    print("No models tested - skipping visualization.")
else:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Updated colors with LLaMA
    colors = {
        'gemma': 'red', 'llama': 'green', 'mistral': 'purple', 
        'pythia': 'blue', 'apertus': 'orange'
    }
    
    # Panel 1: Residual Stream Gains (zoomed to interesting range)
    ax1 = axes[0, 0]
    all_gains_flat = []
    for model_key, res in all_results.items():
        color = colors.get(res['family'], 'gray')
        gains = res['gains']
        layers = range(len(gains))
        ax1.plot(layers, gains, 'o-', label=model_key, color=color, alpha=0.7, markersize=4)
        all_gains_flat.extend(gains[1:])  # Skip initial explosion for ylim
        
        # Mark Paper #2 inversion point if exists
        if res.get('paper2_inversion_layer') is not None:
            l28 = res['paper2_inversion_layer']
            if l28 < len(gains):
                ax1.axvline(x=l28, color=color, linestyle='--', alpha=0.5)
                ax1.annotate(f'L{l28}', xy=(l28, gains[l28]), fontsize=8)
    
    ax1.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='gain=1')
    ax1.set_xlabel('Layer')
    ax1.set_ylabel('Residual Stream Gain')
    ax1.set_title('Residual Stream Gain per Layer (excl. initial)')
    ax1.legend(loc='upper left')
    
    # Dynamic ylim based on data (excluding extreme initial gains)
    if all_gains_flat:
        p5, p95 = np.percentile(all_gains_flat, [5, 95])
        margin = (p95 - p5) * 0.2
        ax1.set_ylim(max(0.5, p5 - margin), min(2.0, p95 + margin))
    
    # Panel 2: Residual Stream Norms
    ax2 = axes[0, 1]
    for model_key, res in all_results.items():
        color = colors.get(res['family'], 'gray')
        norms = res['norms']
        layers = range(len(norms))
        ax2.plot(layers, norms, 's-', label=model_key, color=color, alpha=0.7, markersize=4)
    ax2.set_xlabel('Position (0=embedding)')
    ax2.set_ylabel('Residual Norm')
    ax2.set_title('Residual Stream Norm per Layer')
    ax2.legend()
    
    # Panel 3: Last Layer Gains Comparison (with Mistral)
    ax3 = axes[1, 0]
    models = list(all_results.keys()) + ['mistral-7b (ref)']
    last_gains = [all_results[m]['last_gain'] for m in all_results.keys()] + [MISTRAL_REFERENCE['last_layer_gain']]
    bar_colors = [colors.get(all_results[m]['family'], 'gray') for m in all_results.keys()] + ['purple']
    
    x = np.arange(len(models))
    bars = ax3.bar(x, last_gains, color=bar_colors, alpha=0.7)
    ax3.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='gain=1')
    ax3.set_xlabel('Model')
    ax3.set_ylabel('Last Layer Gain')
    ax3.set_title('Last Layer Expansion (Residual Stream)')
    ax3.set_xticks(x)
    ax3.set_xticklabels(models, rotation=20, ha='right')
    
    # Add value labels
    for bar, val in zip(bars, last_gains):
        ax3.annotate(f'{val:.2f}x', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                     ha='center', va='bottom', fontsize=10)
    
    # Panel 4: Initial vs Last Gain (with MLP type markers)
    ax4 = axes[1, 1]
    
    # Different markers for different MLP types
    mlp_markers = {'GELU': 'o', 'GeGLU': 's', 'SwiGLU': '^', 'xIELU': 'D'}
    
    for model_key, res in all_results.items():
        color = colors.get(res['family'], 'gray')
        marker = mlp_markers.get(res['mlp_type'], 'o')
        ax4.scatter(res['initial_gain'], res['last_gain'], s=200, color=color, 
                   marker=marker, label=f"{model_key} ({res['mlp_type']})", 
                   alpha=0.7, edgecolors='black')
    
    # Add Mistral reference
    ax4.scatter(MISTRAL_REFERENCE['initial_explosion'], MISTRAL_REFERENCE['last_layer_gain'], 
               s=200, color='purple', label='mistral-7b (SwiGLU)', alpha=0.7, 
               edgecolors='black', marker='^')
    
    ax4.axhline(y=1.0, color='black', linestyle='--', alpha=0.3)
    ax4.axvline(x=1.0, color='black', linestyle='--', alpha=0.3)
    ax4.set_xlabel('Initial Gain (Emb→L0)')
    ax4.set_ylabel('Last Layer Gain')
    ax4.set_title('Initial vs Final Expansion (by MLP Type)')
    ax4.legend(fontsize=8, loc='upper right')
    ax4.set_xscale('log')
    
    plt.tight_layout()
    output_path = f'{results_dir}/residual_stream_complete_validation.png'
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved: {output_path}")

In [ ]:
# Cell 8b: Cumulative Energy Visualization
# Product of gains = "Net Inertia" across layers
# This visualizes the total energy transformation from embedding to output

print("="*60)
print("CUMULATIVE ENERGY ANALYSIS")
print("="*60)

if len(all_results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = {
        'gemma': 'red', 'llama': 'green', 'mistral': 'purple', 
        'pythia': 'blue', 'apertus': 'orange'
    }
    
    # Panel 1: Cumulative Energy (log scale product of gains)
    ax1 = axes[0]
    
    for model_key, res in all_results.items():
        color = colors.get(res['family'], 'gray')
        gains = res['gains']
        
        # Compute cumulative product (in log space for numerical stability)
        log_gains = np.log(np.array(gains) + 1e-10)  # Avoid log(0)
        cumulative_log = np.cumsum(log_gains)
        cumulative_energy = np.exp(cumulative_log)
        
        layers = range(len(cumulative_energy))
        ax1.semilogy(layers, cumulative_energy, 'o-', label=model_key, 
                     color=color, alpha=0.7, markersize=4)
        
        # Store for later
        res['cumulative_energy'] = cumulative_energy.tolist()
        res['final_cumulative'] = float(cumulative_energy[-1])
        
        # Print final cumulative energy
        print(f"{model_key}: Final Cumulative Energy = {cumulative_energy[-1]:.2e}")
    
    ax1.set_xlabel('Layer')
    ax1.set_ylabel('Cumulative Energy (∏ gains)')
    ax1.set_title('Cumulative Energy = Product of All Gains\n"Net Inertia" from Embedding to Output')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=1.0, color='black', linestyle='--', alpha=0.3, label='neutral')
    
    # Panel 2: Final Cumulative Energy by Norm Type
    ax2 = axes[1]
    
    # Group by norm type
    norm_groups = {}
    for name, res in all_results.items():
        norm = res['norm_type']
        if norm not in norm_groups:
            norm_groups[norm] = []
        norm_groups[norm].append((name, res['final_cumulative']))
    
    # Add Mistral reference
    # Mistral's cumulative would be ~43.86 * 1.0^29 * 1.37 ≈ 60
    mistral_cumulative = MISTRAL_REFERENCE['initial_explosion'] * MISTRAL_REFERENCE['last_layer_gain']
    if 'RMSNorm' in norm_groups:
        norm_groups['RMSNorm'].append(('mistral-7b (ref)', mistral_cumulative))
    
    # Bar chart
    x_pos = 0
    width = 0.35
    for norm, models in norm_groups.items():
        for name, energy in models:
            bar_color = colors.get(name.split('-')[0], 'gray')
            if 'mistral' in name:
                bar_color = 'purple'
            ax2.bar(x_pos, energy, width, color=bar_color, alpha=0.7, 
                   edgecolor='black', label=name)
            ax2.annotate(f'{energy:.1e}', xy=(x_pos, energy), 
                        ha='center', va='bottom', fontsize=8, rotation=45)
            x_pos += 1
        x_pos += 0.5  # Gap between norm types
    
    ax2.set_ylabel('Final Cumulative Energy')
    ax2.set_title('Total "Net Inertia" by Model\nHigher = More Amplification')
    ax2.set_yscale('log')
    ax2.legend(loc='upper right', fontsize=8)
    
    # Key insight annotation
    ax2.annotate('LayerNorm\nContracts', xy=(0, 1), fontsize=10, ha='center', 
                 xycoords='axes fraction', va='top', color='blue')
    ax2.annotate('RMSNorm\nExpands', xy=(0.7, 1), fontsize=10, ha='center',
                 xycoords='axes fraction', va='top', color='red')
    
    plt.tight_layout()
    output_path = f'{results_dir}/cumulative_energy_analysis.png'
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nSaved: {output_path}")
    
    # Summary table
    print("\n" + "="*60)
    print("CUMULATIVE ENERGY SUMMARY")
    print("="*60)
    print("\n| Model | Norm | MLP | Final Cumulative | Net Effect |")
    print("|-------|------|-----|------------------|------------|")
    for name, res in all_results.items():
        net = "AMPLIFY" if res['final_cumulative'] > 1.0 else "ATTENUATE"
        print(f"| {name} | {res['norm_type']} | {res['mlp_type']} | {res['final_cumulative']:.2e} | {net} |")
    
    print(f"| mistral-7b (ref) | RMSNorm | SwiGLU | {mistral_cumulative:.2e} | AMPLIFY |")
    
    print("\n>>> KEY INSIGHT <<<")
    print("Cumulative Energy = ∏(all layer gains)")
    print("- If > 1: Net AMPLIFICATION from embedding to output")
    print("- If < 1: Net ATTENUATION (information loss)")
    print("- Skip connections PRESERVE signal even when FFN contracts")
else:
    print("No models tested - skipping cumulative energy analysis.")

In [ ]:
# Cell 10: Save Results (v2 with Cumulative Energy)

import os
results_dir = '../Results'
os.makedirs(results_dir, exist_ok=True)

# Compile output
output = {
    'experiment': 'Complete Residual Stream Validation v2',
    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'purpose': 'Validate universal Compress-then-Broadcast pattern with SwiGLU comparison',
    'models_tested': list(all_results.keys()),
    'mistral_reference': MISTRAL_REFERENCE,
    'results': {},
    'swiglu_comparison': {},
    'cumulative_energy': {},
    'universal_principle_test': {},
    'paper2_connection': {},
    'conclusions': {}
}

# Convert results (handle numpy types)
for name, res in all_results.items():
    output['results'][name] = {
        'family': res['family'],
        'norm_type': res['norm_type'],
        'mlp_type': res['mlp_type'],
        'n_layers': res['n_layers'],
        'initial_gain': res['initial_gain'],
        'last_gain': res['last_gain'],
        'last_expands': res['last_expands'],
        'max_gain': res['max_gain'],
        'max_gain_layer': res['max_gain_layer'],
        'contracting_pct': res['contracting_pct'],
        'expanding_pct': res['expanding_pct'],
        'paper2_inversion_layer': res.get('paper2_inversion_layer'),
        'paper2_inversion_gain': res.get('paper2_inversion_gain'),
        'cumulative_energy': res.get('cumulative_energy'),
        'final_cumulative': res.get('final_cumulative'),
        'gains': res['gains'],
        'norms': res['norms']
    }

# SwiGLU comparison (LLaMA 3.1 vs Mistral)
swiglu_models = {n: r for n, r in all_results.items() if r['mlp_type'] == 'SwiGLU'}
if 'llama3.1-8b' in swiglu_models:
    llama_gain = swiglu_models['llama3.1-8b']['last_gain']
    mistral_gain = MISTRAL_REFERENCE['last_layer_gain']
    output['swiglu_comparison'] = {
        'llama31_8b_last_gain': llama_gain,
        'mistral_7b_last_gain': mistral_gain,
        'ratio': llama_gain / mistral_gain,
        'consistent': 0.8 <= (llama_gain / mistral_gain) <= 1.2,
        'interpretation': 'Same MLP type → Similar expansion' if 0.8 <= (llama_gain / mistral_gain) <= 1.2 else 'Divergent behavior'
    }

# Cumulative Energy analysis
for name, res in all_results.items():
    if 'final_cumulative' in res and res['final_cumulative'] is not None:
        output['cumulative_energy'][name] = {
            'final_cumulative': res['final_cumulative'],
            'net_effect': 'AMPLIFY' if res['final_cumulative'] > 1.0 else 'ATTENUATE'
        }

# Universal principle test
all_expand = all(r['last_expands'] for r in all_results.values()) if all_results else False
output['universal_principle_test'] = {
    'all_expand_at_last_layer': all_expand,
    'families_tested': list(set(r['family'] for r in all_results.values())),
    'norms_tested': list(set(r['norm_type'] for r in all_results.values())),
    'mlps_tested': list(set(r['mlp_type'] for r in all_results.values())),
    'total_models': len(all_results) + 1  # +1 for Mistral reference
}

# Paper #2 connection
if 'apertus-8b' in all_results:
    apertus = all_results['apertus-8b']
    l28_gain = apertus.get('paper2_inversion_gain')
    output['paper2_connection'] = {
        'apertus_tested': True,
        'layer28_gain': l28_gain,
        'layer28_expands': l28_gain > 1.0 if l28_gain else None,
        'last_layer_gain': apertus['last_gain'],
        'inversion_is_expansion': l28_gain > 1.0 if l28_gain else None
    }
else:
    output['paper2_connection'] = {'apertus_tested': False}

# Conclusions
rmsnorm_expand = all(r['last_expands'] for r in all_results.values() if r['norm_type'] == 'RMSNorm')
layernorm_contract = all(r['last_gain'] < 1.0 for r in all_results.values() if r['norm_type'] == 'LayerNorm')

output['conclusions'] = {
    'universal_expansion': 'CONFIRMED' if all_expand else 'NORMALIZATION-DEPENDENT',
    'rmsnorm_all_expand': rmsnorm_expand,
    'layernorm_all_contract': layernorm_contract,
    'measurement_insight': 'Residual stream measurement required for accurate gain analysis',
    'skip_connection_role': 'Universal expansion mechanism across all architectures',
    'cumulative_energy_insight': 'Product of gains reveals net information amplification/attenuation',
    'swiglu_finding': output.get('swiglu_comparison', {}).get('interpretation', 'Not tested')
}

output_path = f'{results_dir}/residual_stream_complete_validation_v2_results.json'
with open(output_path, 'w') as f:
    json.dump(output, f, indent=2)
print(f"Saved: {output_path}")

# List all output files
print(f"\nOutput files:")
print(f"  {output_path}")
print(f"  {results_dir}/residual_stream_complete_validation.png")
print(f"  {results_dir}/cumulative_energy_analysis.png")

# Auto-download for Colab
try:
    from google.colab import files
    files.download(output_path)
    files.download(f'{results_dir}/residual_stream_complete_validation.png')
    files.download(f'{results_dir}/cumulative_energy_analysis.png')
    print("\nFiles downloaded!")
except:
    pass

In [ ]:
# Cell 11: THE RLHF SAFETY BRAKE TEST - Base vs Instruct
# This is the definitive test of Gemini's hypothesis!
#
# Hypothesis:
#   - Base model (no RLHF) → EXPANDS like Mistral (~1.37x)
#   - Instruct model (RLHF) → CONTRACTS like measured (0.48x)
#
# If confirmed: RLHF is thermodynamically a "dampening" (energy dissipation) effect!

print("="*60)
print("RLHF SAFETY BRAKE TEST: Base vs Instruct")
print("="*60)

# Define the comparison models
RLHF_TEST_MODELS = {
    'llama3.1-8b-base': {
        'hf_name': 'meta-llama/Llama-3.1-8B',
        'params': 8e9,
        'layers': 32,
        'family': 'llama',
        'norm': 'RMSNorm',
        'mlp': 'SwiGLU',
        'memory_gb': 20,
        'rlhf': False,  # NO RLHF!
        'expected': 'EXPANSION (like Mistral ~1.37x)'
    },
    'llama3.1-8b-instruct': {
        'hf_name': 'meta-llama/Llama-3.1-8B-Instruct',
        'params': 8e9,
        'layers': 32,
        'family': 'llama',
        'norm': 'RMSNorm',
        'mlp': 'SwiGLU',
        'memory_gb': 20,
        'rlhf': True,  # WITH RLHF!
        'expected': 'CONTRACTION (~0.48x as measured)'
    }
}

print("\nTest Configuration:")
print("-" * 50)
for name, info in RLHF_TEST_MODELS.items():
    rlhf_status = "WITH RLHF" if info['rlhf'] else "NO RLHF"
    print(f"  {name}:")
    print(f"    HuggingFace: {info['hf_name']}")
    print(f"    RLHF: {rlhf_status}")
    print(f"    Expected: {info['expected']}")
    print()

print("Hypothesis to test:")
print("  If Base EXPANDS and Instruct CONTRACTS →")
print("  RLHF is proven to be a 'Safety Brake' (energy dampening)")
print()
print("Run this cell, then execute the analysis loop with:")
print("  models_to_run = RLHF_TEST_MODELS")

In [ ]:
# Cell 10: Save Results

import os
results_dir = '../Results'
os.makedirs(results_dir, exist_ok=True)

# Compile output
output = {
    'experiment': 'Complete Residual Stream Validation',
    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'purpose': 'Validate universal Compress-then-Broadcast pattern across architectures',
    'models_tested': list(all_results.keys()),
    'mistral_reference': MISTRAL_REFERENCE,
    'results': {},
    'universal_principle_test': {},
    'paper2_connection': {},
    'conclusions': {}
}

# Convert results (handle numpy types)
for name, res in all_results.items():
    output['results'][name] = {
        'family': res['family'],
        'norm_type': res['norm_type'],
        'mlp_type': res['mlp_type'],
        'n_layers': res['n_layers'],
        'initial_gain': res['initial_gain'],
        'last_gain': res['last_gain'],
        'last_expands': res['last_expands'],
        'max_gain': res['max_gain'],
        'max_gain_layer': res['max_gain_layer'],
        'contracting_pct': res['contracting_pct'],
        'expanding_pct': res['expanding_pct'],
        'paper2_inversion_layer': res.get('paper2_inversion_layer'),
        'paper2_inversion_gain': res.get('paper2_inversion_gain'),
        'gains': res['gains'],
        'norms': res['norms']
    }

# Universal principle test
all_expand = all(r['last_expands'] for r in all_results.values()) if all_results else False
output['universal_principle_test'] = {
    'all_expand_at_last_layer': all_expand,
    'families_tested': list(set(r['family'] for r in all_results.values())),
    'norms_tested': list(set(r['norm_type'] for r in all_results.values())),
    'total_models': len(all_results) + 1  # +1 for Mistral reference
}

# Paper #2 connection
if 'apertus-8b' in all_results:
    apertus = all_results['apertus-8b']
    l28_gain = apertus.get('paper2_inversion_gain')
    output['paper2_connection'] = {
        'apertus_tested': True,
        'layer28_gain': l28_gain,
        'layer28_expands': l28_gain > 1.0 if l28_gain else None,
        'last_layer_gain': apertus['last_gain'],
        'inversion_is_expansion': l28_gain > 1.0 if l28_gain else None
    }
else:
    output['paper2_connection'] = {'apertus_tested': False}

# Conclusions
output['conclusions'] = {
    'universal_expansion': 'CONFIRMED' if all_expand else 'PARTIAL',
    'measurement_insight': 'Residual stream measurement required for accurate gain analysis',
    'skip_connection_role': 'Universal expansion mechanism across all architectures',
    'layernorm_vs_rmsnorm': 'Both show expansion, but RMSNorm hides it in residual stream'
}

output_path = f'{results_dir}/residual_stream_complete_validation_results.json'
with open(output_path, 'w') as f:
    json.dump(output, f, indent=2)
print(f"Saved: {output_path}")

# List all output files
print(f"\nOutput files:")
print(f"  {output_path}")
print(f"  {results_dir}/residual_stream_complete_validation.png")

# Auto-download for Colab
try:
    from google.colab import files
    files.download(output_path)
    files.download(f'{results_dir}/residual_stream_complete_validation.png')
    print("\nFiles downloaded!")
except:
    pass